# Agentic RAG Using CrewAI & LangChain

We are going to see how agents can be involved in the RAG system to rtrieve the most relevant information.

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 sentence-transformers langchain-groq langchain_huggingface --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 8.5 MB/s et

In [2]:
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools  import tool
from crewai import Crew
from crewai import Task
from crewai import Agent

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


## Mention the Groq API Key

In [3]:
import os

# Set the API key
os.environ['GROQ_API_KEY']="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" #SHHHH🤫

## Mention the LLM being used

In [4]:
#Initializing an LLM
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1", #tells the ChatOpenAI class where to find the Groq API
    openai_api_key=os.environ['GROQ_API_KEY'],
    model_name="llama3-8b-8192",
    temperature=0.1, #creativity
    max_tokens=1000,
)

In [5]:
#We need a pdf !!
import requests

pdf_url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
response = requests.get(pdf_url)

with open('attenstion_is_all_you_need.pdf', 'wb') as file:
    file.write(response.content)

In [6]:
#  Get the full path of the PDF
# pdf_path = os.path.join(os.getcwd(), "example.pdf")  # Replace 'example.pdf' with your PDF file name


## Create a RAG tool variable to pass our PDF

In [31]:
#creating a RAG tool
rag_tool = PDFSearchTool(pdf='attenstion_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",

            ),
        ),
        #To Generate Embeddings
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)
#dont worry if you find this confusing .... we're just setting up our agent and configuring the LLM
#You can always refer documentations on inteternt for the syntax :) <3

BAAI : https://huggingface.co/BAAI/bge-large-zh

In [8]:
rag_tool.run("What is this pdf about ??") #calling rag tool to test

Using Tool: Search a PDF's content


'Relevant Content:\nTable 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base model. All metrics are on the English-to-German translation development set, newstest2013. Listed perplexities are per-wordpiece, according to our byte-pair encoding, and should not be compared to per-word perplexities. N d modeldffh d kdvPdropϵlstrain PPL BLEU params steps (dev) (dev)×106 base 6 512 2048 8 64 64 0.1 0.1 100K 4.92 25.8 65 (A)1 512 512 5.29 24.9 4 128 128 5.00 25.5 16 32 32 4.91 25.8 32 16 16 5.01 25.4 (B)16 5.16 25.1 58 32 5.01 25.4 60 (C)2 6.11 23.7 36 4 5.19 25.3 50 8 4.88 25.5 80 256 32 32 5.75 24.5 28 1024 128 128 4.66 26.0 168 1024 5.12 25.4 53 4096 4.75 26.2 90 (D)0.0 5.77 24.6 0.2 4.95 25.5 0.0 4.67 25.3 0.2 5.47 25.7 (E) positional embedding instead of sinusoids 4.92 25.7 big 6 1024 4096 16 0.3 300K 4.33 26.4 213 In Table 3 rows (B), we observe that reducing the attention key size dkhurts model quality. This suggests that determining compa

## Mention the Tavily API Key
Internet search

In [32]:
import os

# Set the Tavily API key
os.environ['TAVILY_API_KEY']="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"

In [33]:
web_search_tool = TavilySearchResults(k=3)

In [34]:
# web_search_tool.run

**Tools are the "how" -** they provide the functionalities.

**Agents are the "who" -** they make the decisions.

**Tasks are the "what" -** they define the goals.

---



## Define a Tool

In [35]:
@tool
#  tool to indentify which tools to call
def router_tool(question):
  """Router Function"""
  if 'self-attention' in question:
    return 'vectorstore'
  else:
    return 'web_search'

## Create Agents to work with

In [36]:
# Intelligent agent, can understand the context and intent of the question better than simple keyword matching.
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True, #to provide info about thought process
  allow_delegation=False, #use only the tools provided
  llm=llm,
)

In [37]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [38]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [39]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [40]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

## Defining Tasks

Agent: Who will do ||
Task: What needs to be done

In [41]:
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [42]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],

)

In [43]:
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [44]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,

)

## Define a flow for the use case

In [45]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)

## Ask any query

In [23]:
inputs ={"question":"Whats capital of india ??"}

## Kick off the agent pipeline

In [46]:
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Whats attention mechanism described in the pdf ?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is asking about an attention mechanism described in a pdf, which seems to be related to Retrieval-Augmented Generation.
Action: router_tool
Action Input: {"question": "Whats attention mechanism described in the pdf ?", "related_topics": ["Retrieval-Augmented Generation"]} 

I encountered an error while trying to use the tool. This was the error: router_tool() got an unexpected keyword argument 'related_topics'.
 Tool router_tool accepts these inputs: router_tool() - Router Function

Thought: I wi

## Obtain the final response

In [47]:
print(result)

The attention mechanism described in the PDF is a technique used in deep learning models to focus on specific parts of the input data that are relevant to the task at hand. It allows the model to selectively weigh the importance of different input features or tokens, which can improve the model's performance and accuracy.


### convert op to markdown file using agent

In [48]:
from IPython.display import Markdown, display


output_text = result

# Convert to Markdown and display
display(Markdown(output_text))

# Save to a file (optional)
with open('output.md', 'w') as f:
    f.write(output_text)

The attention mechanism described in the PDF is a technique used in deep learning models to focus on specific parts of the input data that are relevant to the task at hand. It allows the model to selectively weigh the importance of different input features or tokens, which can improve the model's performance and accuracy.

Looks better !

In [49]:
inputs ={"question":"Whats attention mechanism described in the pdf ?"}

## Kick off the agent pipeline

In [28]:
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Whats attention mechanism described in the pdf ?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is asking about an attention mechanism described in a pdf, which seems to be related to Retrieval-Augmented Generation.
Action: router_tool
Action Input: {"question": "Whats attention mechanism described in the pdf ?", "related_topics": ["Retrieval-Augmented Generation"]} 

I encountered an error while trying to use the tool. This was the error: router_tool() got an unexpected keyword argument 'related_topics'.
 Tool router_tool accepts these inputs: router_tool() - Router Function

Thought: The 

## Obtain the final response

In [50]:
  print(result)

The attention mechanism described in the PDF is a technique used in deep learning models to focus on specific parts of the input data that are relevant to the task at hand. It allows the model to selectively weigh the importance of different input features or tokens, which can improve the model's performance and accuracy.


### convert op to markdown file using agent

In [51]:
from IPython.display import Markdown, display


output_text = result

# Convert to Markdown and display
display(Markdown(output_text))

# Save to a file (optional)
with open('output.md', 'w') as f:
    f.write(output_text)

The attention mechanism described in the PDF is a technique used in deep learning models to focus on specific parts of the input data that are relevant to the task at hand. It allows the model to selectively weigh the importance of different input features or tokens, which can improve the model's performance and accuracy.